In [ ]:
# %% JUPYTER CELL: Run SSL DINOv2 on selected samples (e.g. 37-110)

from pathlib import Path
import os, glob, os.path as op
import sys

def _detect_user_base():
    aau = glob.glob("/work/Member Files:*")
    if aau:
        return op.basename(aau[0])
    sdu = [d for d in glob.glob("/work/*#*") if op.isdir(d)]
    return op.basename(sdu[0]) if sdu else None

USER_BASE_DIR = os.environ.get("USER_BASE_DIR") or _detect_user_base()
if not USER_BASE_DIR:
    raise RuntimeError("Could not determine USER_BASE_DIR")
os.environ["USER_BASE_DIR"] = USER_BASE_DIR

WORK_ROOT = Path("/work") / USER_BASE_DIR

SSL_TRAINING_ROOT = WORK_ROOT / "CellScanData" / "Zoom10x - Quality Assessment_Cleaned"

SSL_FIRST_SAMPLE = 37
SSL_LAST_SAMPLE  = 109

if SSL_FIRST_SAMPLE > SSL_LAST_SAMPLE:
    raise ValueError(f"SSL_FIRST_SAMPLE ({SSL_FIRST_SAMPLE}) > SSL_LAST_SAMPLE ({SSL_LAST_SAMPLE})")

os.environ["SSL_TRAINING_ROOT"] = SSL_TRAINING_ROOT.as_posix()
os.environ["SSL_FIRST_SAMPLE"] = str(SSL_FIRST_SAMPLE)
os.environ["SSL_LAST_SAMPLE"]  = str(SSL_LAST_SAMPLE)

# Optional: if Stat_Dataset is moved elsewhere
# os.environ["STAT_DATASETS_ROOT"] = "/work/projects/myproj/SOME/OTHER/PATH"

os.environ.setdefault("SSL_PATCH_WORKERS", "14")
os.environ.setdefault("SSL_MIN_TOTAL_CROPS", "10000")
os.environ.setdefault("SSL_FAIL_ON_MISSING_SAMPLES", "0")
os.environ.setdefault("SSL_LIGHTLY_OUT_DIR", f"outputLightly_ssl_{SSL_FIRST_SAMPLE}_{SSL_LAST_SAMPLE}")
os.environ.setdefault("SSL_PREVIEW_ONLY", "0")
os.environ.setdefault("SSL_PROGRESS_EVERY_IMAGES", "10")
os.environ.setdefault("SSL_PROGRESS_EVERY_FILES", "2000")

print("USER_BASE_DIR:", USER_BASE_DIR)
print("SSL_TRAINING_ROOT:", SSL_TRAINING_ROOT)
print(f"SSL sample range: {SSL_FIRST_SAMPLE}-{SSL_LAST_SAMPLE}")
print("SSL_MIN_TOTAL_CROPS:", os.environ.get("SSL_MIN_TOTAL_CROPS"))
print("SSL_FAIL_ON_MISSING_SAMPLES:", os.environ.get("SSL_FAIL_ON_MISSING_SAMPLES"))
print("SSL_LIGHTLY_OUT_DIR:", os.environ.get("SSL_LIGHTLY_OUT_DIR"))
print("SSL_PREVIEW_ONLY:", os.environ.get("SSL_PREVIEW_ONLY"))
print("SSL_PROGRESS_EVERY_IMAGES:", os.environ.get("SSL_PROGRESS_EVERY_IMAGES"))
print("SSL_PROGRESS_EVERY_FILES:", os.environ.get("SSL_PROGRESS_EVERY_FILES"))

existing_samples = []
missing_samples = []
for i in range(SSL_FIRST_SAMPLE, SSL_LAST_SAMPLE + 1):
    d = SSL_TRAINING_ROOT / f"Sample {i}"
    if d.is_dir():
        existing_samples.append(i)
    else:
        missing_samples.append(i)

print(f"Found {len(existing_samples)} Sample folders in range.")
if missing_samples:
    print("Missing Sample folders (will contribute 0 images unless fail-on-missing is enabled):", missing_samples)

progress_root = SSL_TRAINING_ROOT.parent.as_posix()
print("Progress file pattern:", f"{progress_root}/SSL_LightlySelection_224crops_*/selection_progress.json")
print("Monitor command (run in another terminal while training runs):")
print(
    f"latest=$(ls -dt {progress_root}/SSL_LightlySelection_224crops_*/selection_progress.json 2>/dev/null | head -1); "
    "echo $latest; [ -n \"$latest\" ] && watch -n 2 cat \"$latest\""
)

%cd /work/projects/myproj

!{sys.executable} -u LightlyTrainSSL.py
